In [113]:
import pandas as pd
import numpy as np

In [114]:
Political_Data = pd.read_csv('Data/Clean/Combined/Political-Data.csv')
Political_Data.head()

,Country,Year,Gov,BC,PI,RPE,PR,CL,Urban,Fertility
0,Algeria,1972,-9,0.0,0,2.742,6.0,6.0,39.831,7.546
1,Algeria,1973,-9,0.0,0,2.642,6.0,6.0,39.997,7.491
2,Algeria,1974,-9,0.0,0,2.222,6.0,6.0,40.163,7.440
3,Algeria,1975,-9,6.0,250,2.711,7.0,7.0,40.330,7.373
4,Algeria,1976,-9,6.0,0,2.431,6.0,6.0,40.497,7.306


In [115]:
# Create DataFrame
df_01 = pd.DataFrame(Political_Data)
df_01.dtypes

Country       object
Year           int64
Gov            int64
BC           float64
PI            object
RPE          float64
PR           float64
CL           float64
Urban        float64
Fertility    float64
dtype: object

In [116]:
# Change PI to numeric
df_01['PI'] = pd.to_numeric(df_01['PI'], errors='coerce')
df_01.dtypes

Country       object
Year           int64
Gov            int64
BC           float64
PI           float64
RPE          float64
PR           float64
CL           float64
Urban        float64
Fertility    float64
dtype: object

In [117]:
# Filter the DataFrame for years 2000 to 2020 without using the index
df_01 = df_01[(df_01['Year'] >= 1975)]

In [118]:
df_01.head()

,Country,Year,Gov,BC,PI,RPE,PR,CL,Urban,Fertility
3,Algeria,1975,-9,6.0,250.0,2.711,7.0,7.0,40.330,7.373
4,Algeria,1976,-9,6.0,0.0,2.431,6.0,6.0,40.497,7.306
5,Algeria,1977,-9,6.0,0.0,2.643,6.0,6.0,40.928,7.230
6,Algeria,1978,-9,6.0,0.0,2.587,6.0,6.0,41.794,7.140
7,Algeria,1979,-9,6.0,0.0,2.487,6.0,6.0,42.665,7.059


In [119]:
# Define the starting year for intervals
start_year = 1975

# Calculate the interval group
df_01['Interval_Group'] = ((df_01['Year'] - start_year) // 5) * 5 + start_year

# Group by 'ID' and 'Interval_Group', then calculate the mean
df_01 = df_01.groupby(['Country', 'Interval_Group']).mean(numeric_only=True)

In [120]:
df_01.head()

Year  Gov   BC      PI     RPE   PR   CL     Urban  \
Country Interval_Group                                                          
Algeria 1975            1977.00 -9.0  6.0    50.0  2.5718  6.2  6.2  41.24280   
        1980            1982.25 -9.0  3.0     0.0  2.2750  6.0  6.0  45.52825   
        1985            1987.00 -7.6  3.4   175.0  2.5132  5.8  5.8  49.67180   
        1990            1992.00 -5.0  2.0  2062.2  1.4426  5.8  5.8  53.65420   
        1995            1997.00 -3.0  0.8  3449.8  1.2556  6.0  6.0  57.54920   

                        Fertility  
Country Interval_Group             
Algeria 1975               7.2216  
        1980               6.5605  
        1985               5.3624  
        1990               4.2208  
        1995               2.9650

In [121]:
# Reset the index 
df_01 = df_01.reset_index()

In [122]:
# Drop the 'Year' column
df_01 = df_01.drop('Year', axis=1)

In [123]:
# Rename the 'Interval_Group' column to 'Year'
df_01 = df_01.rename(columns={'Interval_Group': 'Year'})

df_01.head()

,Country,Year,Gov,BC,PI,RPE,PR,CL,Urban,Fertility
0,Algeria,1975,-9.0,6.0,50.0,2.5718,6.2,6.2,41.24280,7.2216
1,Algeria,1980,-9.0,3.0,0.0,2.2750,6.0,6.0,45.52825,6.5605
2,Algeria,1985,-7.6,3.4,175.0,2.5132,5.8,5.8,49.67180,5.3624
3,Algeria,1990,-5.0,2.0,2062.2,1.4426,5.8,5.8,53.65420,4.2208
4,Algeria,1995,-3.0,0.8,3449.8,1.2556,6.0,6.0,57.54920,2.9650


In [124]:
# Lag 1 Time Frame
# Create DataFrame
df_Avg_01 = pd.DataFrame(df_01)

# Set both 'Year' and 'Country' as indices
df_Avg_01.set_index(['Year', 'Country'], inplace=True)

# Lag DV by 1 year
df_Avg_01['Gov'] = df_Avg_01['Gov'].shift(1)
df_Avg_01['BC'] = df_Avg_01['BC'].shift(1)
df_Avg_01['RPE'] = df_Avg_01['RPE'].shift(1)
df_Avg_01['PR'] = df_Avg_01['PR'].shift(1)
df_Avg_01['CL'] = df_Avg_01['CL'].shift(1)
df_Avg_01['Urban'] = df_Avg_01['Urban'].shift(1)
df_Avg_01['Fertility'] = df_Avg_01['Fertility'].shift(1)
df_Avg_01.head()


,,Gov,BC,PI,RPE,PR,CL,Urban,Fertility
Year,Country,,,,,,,,
1975,Algeria,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN
1980,Algeria,-9.0,6.0,0.0,2.5718,6.2,6.2,41.24280,7.2216
1985,Algeria,-9.0,3.0,175.0,2.2750,6.0,6.0,45.52825,6.5605
1990,Algeria,-7.6,3.4,2062.2,2.5132,5.8,5.8,49.67180,5.3624
1995,Algeria,-5.0,2.0,3449.8,1.4426,5.8,5.8,53.65420,4.2208


In [125]:
# Reset the index and keep the 'Year' as a column
df_Avg_01 = df_Avg_01.reset_index()

In [126]:
df_Avg_01.columns

Index(['Year', 'Country', 'Gov', 'BC', 'PI', 'RPE', 'PR', 'CL', 'Urban',
       'Fertility'],
      dtype='object')

In [127]:
# Filter the DataFrame for years 2000 to 2020 without using the index
df_Avg_01 = df_Avg_01[(df_Avg_01['Year'] >= 2000) & (df_Avg_01['Year'] <= 2020)]
df_Avg_01.head()

,Year,Country,Gov,BC,PI,RPE,PR,CL,Urban,Fertility
5,2000,Algeria,-3.0,0.8,1975.0,1.2556,6.0,6.0,57.5492,2.9650
6,2005,Algeria,-2.0,0.0,1000.0,1.1630,6.0,6.0,61.4954,2.5044
7,2010,Algeria,2.0,0.0,523.0,1.3142,6.0,6.0,65.3388,2.6804
13,2000,Angola,-38.8,6.6,1750.0,1.0574,6.0,6.0,46.5294,6.7946
14,2005,Angola,-3.6,5.4,0.0,1.2586,6.0,6.0,52.4588,6.5678


In [128]:
# Drop 'Country' and 'Year' columns
# df_Avg_01 = df_Avg_01.drop(['Country', 'Year'], axis=1)
df_Avg_01.head()

,Year,Country,Gov,BC,PI,RPE,PR,CL,Urban,Fertility
5,2000,Algeria,-3.0,0.8,1975.0,1.2556,6.0,6.0,57.5492,2.9650
6,2005,Algeria,-2.0,0.0,1000.0,1.1630,6.0,6.0,61.4954,2.5044
7,2010,Algeria,2.0,0.0,523.0,1.3142,6.0,6.0,65.3388,2.6804
13,2000,Angola,-38.8,6.6,1750.0,1.0574,6.0,6.0,46.5294,6.7946
14,2005,Angola,-3.6,5.4,0.0,1.2586,6.0,6.0,52.4588,6.5678


In [129]:
# DF Lag 1 Year
df_Avg_01.to_csv('Data/Clean/Final/Political-Avg-01.csv', index = False)

In [130]:
# DF Lag 2 Avg
# Create DataFrame
df_Avg_02 = pd.DataFrame(df_01)

# Set both 'Year' and 'Country' as indices
df_Avg_02.set_index(['Year', 'Country'], inplace=True)

# Lag DV by 1 year
df_Avg_02['Gov'] = df_Avg_02['Gov'].shift(1)
df_Avg_02['BC'] = df_Avg_02['BC'].shift(1)
df_Avg_02['RPE'] = df_Avg_02['RPE'].shift(1)
df_Avg_02['PR'] = df_Avg_02['PR'].shift(1)
df_Avg_02['CL'] = df_Avg_02['CL'].shift(1)
df_Avg_02['Urban'] = df_Avg_02['Urban'].shift(1)
df_Avg_02['Fertility'] = df_Avg_02['Fertility'].shift(1)
df_Avg_02.head()

,,Gov,BC,PI,RPE,PR,CL,Urban,Fertility
Year,Country,,,,,,,,
1975,Algeria,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN
1980,Algeria,-9.0,6.0,0.0,2.5718,6.2,6.2,41.24280,7.2216
1985,Algeria,-9.0,3.0,175.0,2.2750,6.0,6.0,45.52825,6.5605
1990,Algeria,-7.6,3.4,2062.2,2.5132,5.8,5.8,49.67180,5.3624
1995,Algeria,-5.0,2.0,3449.8,1.4426,5.8,5.8,53.65420,4.2208


In [131]:
# Reset the index and keep the 'Year' as a column
df_Avg_02 = df_Avg_02.reset_index()

In [132]:
df_Avg_02.columns

Index(['Year', 'Country', 'Gov', 'BC', 'PI', 'RPE', 'PR', 'CL', 'Urban',
       'Fertility'],
      dtype='object')

In [133]:
# Filter the DataFrame for years 2000 to 2020 without using the index
df_Avg_02 = df_Avg_02[(df_Avg_02['Year'] >= 2000) & (df_Avg_02['Year'] <= 2020)]
df_Avg_02.head()

,Year,Country,Gov,BC,PI,RPE,PR,CL,Urban,Fertility
5,2000,Algeria,-3.0,0.8,1975.0,1.2556,6.0,6.0,57.5492,2.9650
6,2005,Algeria,-2.0,0.0,1000.0,1.1630,6.0,6.0,61.4954,2.5044
7,2010,Algeria,2.0,0.0,523.0,1.3142,6.0,6.0,65.3388,2.6804
13,2000,Angola,-38.8,6.6,1750.0,1.0574,6.0,6.0,46.5294,6.7946
14,2005,Angola,-3.6,5.4,0.0,1.2586,6.0,6.0,52.4588,6.5678


In [134]:
# Drop 'Country' and 'Year' columns
# df_Avg_02 = df_Avg_02.drop(['Country', 'Year'], axis=1)
df_Avg_02.head()

,Year,Country,Gov,BC,PI,RPE,PR,CL,Urban,Fertility
5,2000,Algeria,-3.0,0.8,1975.0,1.2556,6.0,6.0,57.5492,2.9650
6,2005,Algeria,-2.0,0.0,1000.0,1.1630,6.0,6.0,61.4954,2.5044
7,2010,Algeria,2.0,0.0,523.0,1.3142,6.0,6.0,65.3388,2.6804
13,2000,Angola,-38.8,6.6,1750.0,1.0574,6.0,6.0,46.5294,6.7946
14,2005,Angola,-3.6,5.4,0.0,1.2586,6.0,6.0,52.4588,6.5678


In [135]:
# DF Lag 5 Year
df_Avg_02.to_csv('Data/Clean/Final/Political-Avg-02.csv', index = False)